# Fine-tuning the text classification model

In [3]:
# Imports
import os
import numpy as np
from datasets import load_dataset, load_metric
from tqdm import tqdm
import torch
import re
import evaluate

from transformers import (
    Trainer,
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    TrainingArguments
)

In [4]:
# Check if GPU and CUDA are available.
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce RTX 2080 SUPER


In [5]:
# Settings
model_checkpoint = "distilbert-base-uncased"

# Data preparation

In [9]:
dataset = load_dataset("imdb", trust_remote_code=True)
dataset.pop("unsupervised")

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [12]:
print(f"DATASET TYPE: {type(dataset)}")
print(f"DATASET INFO: {dataset.items()}")
print(f"DATASET COL NAMES: {dataset.column_names}")
print(f"DATASET EXAMPLE: \n {dataset['train'][2]}")

DATASET TYPE: <class 'datasets.dataset_dict.DatasetDict'>
DATASET INFO: dict_items([('train', Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})), ('test', Dataset({
    features: ['text', 'label'],
    num_rows: 25000
}))])
DATASET COL NAMES: {'train': ['text', 'label'], 'test': ['text', 'label']}
DATASET EXAMPLE: 
 {'text': "If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />", 'label': 0}


In [32]:
for i in range(0, 3):
    print(dataset["test"][i])

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [33]:
cleared_text = re.sub('<[^<]+?>', '', dataset["train"][1]["text"])
cleared_text

'"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the s

In [34]:
def clear_text(text: str) -> str:
    return re.sub('<[^<]+?>', '', text)

# Tokenizer

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [36]:
def preprocess_function(examples):    
    inputs = [clear_text(ex) for ex in examples["text"]]
    
    model_inputs = tokenizer(
        inputs, truncation=True
    )

    return model_inputs

In [37]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:03<00:00, 6582.32 examples/s]


In [38]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
})

# Finetuning

In [49]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [50]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [53]:
finetuned_ids = [0, 1, 2, 3, 4]
learning_rates = [2e-3, 2e-4, 2e-5, 2e-7, 2e-9]
n_epochs = [2, 3, 3, 5, 7]

In [54]:
for f_id, lr, n_e in zip(finetuned_ids, learning_rates, n_epochs):
    
    print(f"ID: {f_id}, LEARNING_RATE: {lr}, N_EPOCHS: {n_e}")

    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
    )    

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    training_args = TrainingArguments(
        output_dir=f"{model_checkpoint}-finetuned-CLASSIFICATION-{f_id}",
        learning_rate=lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=n_e,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()

ID: 0, LEARNING_RATE: 0.002, N_EPOCHS: 2


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693400,0.693216,0.500000
2,0.693200,0.693148,0.500000


Checkpoint destination directory distilbert-base-uncased-finetuned-CLASSIFICATION-0/checkpoint-3125 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory distilbert-base-uncased-finetuned-CLASSIFICATION-0/checkpoint-6250 already exists and is non-empty.Saving will proceed but saved results may be invalid.


ID: 1, LEARNING_RATE: 0.0002, N_EPOCHS: 3


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.486200,0.428747,0.865120
2,0.335500,0.304453,0.880960
3,0.310100,0.286176,0.886280
4,0.300000,0.283234,0.888080
5,0.277700,0.282083,0.889920


ID: 4, LEARNING_RATE: 2e-09, N_EPOCHS: 7


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.694900,0.694164,0.497280
2,0.694300,0.694039,0.498800
3,0.694100,0.693954,0.499840
4,0.694100,0.693900,0.500040
5,0.695900,0.693871,0.500200
6,0.695200,0.693861,0.500240
7,0.693900,0.693859,0.500240


# Using the model

In [78]:
text = "This was great movie!"

In [79]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

my_checkpoint = "distilbert-base-uncased-finetuned-CLASSIFICATION-2/checkpoint-9375"

tokenizer = AutoTokenizer.from_pretrained(my_checkpoint)
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained(my_checkpoint)
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

In [80]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'